### Imports

In [5]:
import os
from PIL import Image
import pandas as pd
import torch
from torch.utils.data import Dataset
from torchvision import transforms


In [7]:
class CustomDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the CSV file with target values.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.df = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, str(self.df.iloc[idx, 0]))
        image = Image.open(img_name)

        # Assuming the target column is named 'loss_percentage'
        target = torch.tensor(self.df.iloc[idx, 1], dtype=torch.float32)

        if self.transform:
            image = self.transform(image)

        return image, target

# Specify the paths
csv_file = 'filtered_data.csv'
root_dir = 'SolarPanelSoilingImageDataset\\Solar_Panel_Soiling_Image_dataset\\Pytorch_data'

# Define a transformation (resize, normalize, etc.) if needed
#transform = transforms.Compose([
#    transforms.Resize((224, 224)),
#    transforms.ToTensor(),
#])

# Create an instance of the CustomDataset
custom_dataset = CustomDataset(csv_file=csv_file, root_dir=root_dir)

# Access an example from the dataset
sample_image, sample_target = custom_dataset[0]
print(f'Sample Image Shape: {sample_image.shape}')
print(f'Sample Target: {sample_target}')

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\tfrod\\Desktop\\CV Project\\SolarPanelSoilingImageDataset\\Solar_Panel_Soiling_Image_dataset\\Pytorch_data\\6'